In [1]:
import pandas as pd
import sys
import os
import numpy as np
import cupy as cp
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.kernel_ridge import KernelRidge
from timeit import default_timer as timer 
from sys import getsizeof

print("x")

x


In [2]:
def pivoted_cholesky(matrix, max_rank, diag_rtol=1e-3, name=None):

    if max_rank > np.linalg.matrix_rank(matrix):
            max_rank = np.linalg.matrix_rank(matrix)
    
    matrix_shape = np.asarray(matrix.shape)
    matrix_diag = cp.diagonal(matrix)
    orig_error = cp.amax(matrix_diag)
    max_rank = cp.asnumpy(max_rank)

    def body(m, pchol, perm, matrix_diag):

        maxi = np.argmax(matrix_diag[perm[m:]]) + m

        maxval = matrix_diag[perm][maxi]
        perm[m], perm[maxi] = perm[maxi], perm[m]

        row = matrix[perm[m]][perm[m + 1:]]

        # row = cp.array(row)
        pchol = cp.asnumpy(pchol)

        
        def batch_gather(params, indices, axis=-1):
            return cp.take(params, indices, axis=axis)
        

        prev_rows = pchol[..., :m, :]
        prev_rows_perm_m_onward = batch_gather(prev_rows, perm[..., m + 1:])
        prev_rows_pivot_col = batch_gather(prev_rows, perm[..., m:m + 1])
        
        row -= cp.sum(prev_rows_perm_m_onward * prev_rows_pivot_col, axis = -2)
        pivot = np.sqrt(maxval)

        row /= pivot
        row = np.concatenate(([pivot], row), -1)
        
        matrix_diag_copy = matrix_diag.copy()
        matrix_diag_copy[perm[m:]] -= row**2
        
        pchol[m, perm[m:]] = row
        pchol_shape = pchol.shape

        return m + 1, pchol, perm, matrix_diag_copy

    m = cp.int64(0)

    pchol = cp.zeros(matrix_shape, dtype=matrix.dtype)[..., :max_rank, :]


    perm = np.broadcast_to(
        range(matrix_shape[-1]), matrix_shape[:-1])

    perm = perm.copy()

    def check_cond(m, pchol, perm, matrix_diag):
        del pchol
        del perm
        error = cp.linalg.norm(matrix_diag, ord=1, axis=-1)
        max_error = cp.amax(error / orig_error)
        return (m >= max_rank)

    while(1):
        m, pchol, perm, matrix_diag = body(m, pchol, perm, matrix_diag)
        if check_cond(m, pchol, perm, matrix_diag):
            break

    pchol = cp.transpose(pchol)
    pchol = cp.array(pchol)
    print(type(pchol))

    return pchol


In [ ]:
# import pandas as pd
# energy_set = pd.read_csv('energy.csv')

# energy_set.shape


# X = pd.read_csv('energy.csv', usecols=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8'])
# y = pd.read_csv('energy.csv', usecols=['Y1'])

# X = X.values
# y = y.values

start_load = timer()

X = np.load('data/md17_X.npy')

y = np.load('data/md17_Y.npy')

print("GPU load data time = ", timer() - start_load)

X.shape, y.shape

In [5]:
newfp = np.memmap('data/md17_X.npy', dtype='float32', mode='r', shape=(3,4))

newfp

X.shape, y.shape

NameError: name 'X' is not defined

In [5]:
X = X[:20000]
y= y[:20000]

In [42]:
def execute_krr(lr, ridge_parameter, X_train, y_train, X_test):

    lr_approx_ridge = lr + cp.identity(len(X_train)) * ridge_parameter
    
    print(type(lr_approx_ridge), np.linalg.matrix_rank(lr_approx_ridge), type(y_train))
    alpha = cp.linalg.solve(lr_approx_ridge, y_train)

    y_prediction = []
    
    X_train, X_test = cp.asnumpy(X_train), cp.asnumpy(X_test)
    for j in range(len(X_test)):
        x_test_val = X_test[j].reshape(1,-1)
        pred = 0
        for i in range(len(X_train)):
            x_train_val = X_train[i].reshape(1,-1)
            similarity = cp.array(sklearn.metrics.pairwise.laplacian_kernel(x_train_val, x_test_val))     
            temp = alpha[i][0] * similarity[0]     
            pred += temp[0]
        y_prediction.append(pred)


#     y_predarray = np.array(y_prediction).reshape(-1,1)
#     y_predarray.shape
    print(type(y_prediction))
    return cp.asarray(y_prediction)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

kernel_matrix = sklearn.metrics.pairwise.laplacian_kernel(X_train, Y=None, gamma=None)

kernel_matrix.shape

kernel_matrix.shape, np.linalg.norm(kernel_matrix)


# kernel_matrix = cp.array(kernel_matrix)
# type(kernel_matrix)

In [36]:
start_piv_gpu = timer()


lr = pivoted_cholesky(
        kernel_matrix, 90) 
# print(type(lr))
# print("rank = ", cp.linalg.matrix_rank(lr))

print("GPU piv = ", timer() - start_piv_gpu)

<class 'cupy.core.core.ndarray'>
GPU piv =  0.1648433916270733


In [24]:
start = timer()

lr_t = cp.transpose(lr)

lr_approx = cp.matmul(lr, lr_t)

# print(lr_approx.shape, type(lr_approx))

# print("orig norm", np.linalg.norm(kernel_matrix))
# print("approx norm = ", np.linalg.norm(lr_approx), "approx_rank = ", np.linalg.matrix_rank(lr_approx))


print("GPU piv = ", timer() - start)

getsizeof(kernel_matrix), getsizeof(lr_approx), getsizeof(lr), getsizeof(lr_t)

GPU piv =  0.00038227997720241547


(3819968, 160, 160, 160)

In [11]:


start_krr = timer()

X_train_cp, y_train_cp, X_test_cp, y_test_cp, kernel_matrix_cp = cp.array(X_train), cp.array(y_train), cp.array(X_test), cp.array(y_test), cp.array(kernel_matrix)

y_predarray = execute_krr(kernel_matrix_cp, 0, X_train_cp, y_train_cp, X_test_cp)

print("GPU krr = ", timer() - start_krr)



<class 'cupy.core.core.ndarray'> 691 <class 'cupy.core.core.ndarray'>
<class 'list'>
GPU krr =  13.627636002376676


In [215]:
y_predarray = cp.asnumpy(y_predarray)

In [16]:
krr_laplacian = KernelRidge(alpha=0, kernel='laplacian')
# X_train_stan, y_train_stan = cp.asnumpy(X_train), cp.asnumpy(y_train)
# X_test_stan, y_test_stan = cp.asnumpy(X_test), cp.asnumpy(y_test)
krr_laplacian.fit(X_train, y_train)


stan_pred = krr_laplacian.predict(X_test)
# rsme_krr_standard = np.sqrt(mean_squared_error(y_test_stan, stan_pred))

rsme_krr_standard = np.sqrt(mean_squared_error(y_test, stan_pred))
rsme_krr_manual = np.sqrt(mean_squared_error(y_test, cp.asnumpy(y_predarray)))
    
print("rsme_sk = ", rsme_krr_standard, "rsme_piv = ", rsme_krr_manual)

print(y_test.shape)
for y1, y2, y3 in zip(stan_pred, y_predarray, y_test):
    print(y1, "    ", y2, "     ", y3)
    


rsme_sk =  0.22016128979640934 rsme_piv =  0.22016128979640234
(77, 1)
[42.50418625]      42.50418625223657       [42.49]
[18.74880219]      18.74880219056279       [18.46]
[26.5956469]      26.59564689743978       [26.47]
[16.57089372]      16.57089372351045       [16.66]
[14.54342786]      14.543427855509794       [14.4]
[11.4854897]      11.485489699505427       [11.53]
[24.35788681]      24.357886812106095       [24.23]
[42.51428793]      42.51428792889314       [42.77]
[28.49461779]      28.49461779202789       [28.05]
[24.40258802]      24.40258802227696       [24.6]
[14.0919701]      14.091970102270013       [13.68]
[25.09086759]      25.090867591477117       [25.17]
[19.40841162]      19.40841161554621       [19.42]
[32.62518706]      32.62518705644466       [32.73]
[16.5151109]      16.51511090050267       [16.44]
[9.95767105]      9.957671051062134       [10.38]
[14.54287681]      14.542876805128067       [14.61]
[37.31916618]      37.31916618332013       [36.71]
[43.28484288